In [51]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [52]:
movies = pd.read_csv('dataset.csv')

In [53]:

movies = movies[['id', 'title', 'overview', 'genre']]




In [54]:

# Combine relevant features
movies['features'] = movies['overview'] + ' ' + movies['genre']


In [55]:

# Replace missing values with an empty string
movies['features'] = movies['features'].fillna('')


In [56]:

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['features'])



In [57]:
# Apply Non-Negative Matrix Factorization (NMF)
n_components = 50  # Set the desired number of latent factors
nmf = NMF(n_components=n_components)
latent_matrix = nmf.fit_transform(tfidf_matrix)


In [58]:

# Calculate the cosine similarity between latent factors
latent_similarity = cosine_similarity(latent_matrix)



In [59]:
# Calculate the cosine similarity between latent factors
latent_similarity = cosine_similarity(latent_matrix)

# Function to recommend movies based on similarity
def recommend_movies(movie_title, top_n=5):
    # Get the index of the movie
    idx = movies[movies['title'] == movie_title].index[0]
    
    # Get the pairwise similarity scores
    similarity_scores = list(enumerate(latent_similarity[idx]))
    
    # Sort movies based on similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get top similar movies
    top_movies = [movies.iloc[i[0]]['title'] for i in similarity_scores[1:top_n+1]]
    
    return top_movies

# Example usage for NMF recommendation
recommendations_nmf = recommend_movies('Iron Man', top_n=5)
print(recommendations_nmf)

# Save movies and latent_similarity to pickle files
pickle.dump(movies, open('moviess.pkl', 'wb'))
pickle.dump(latent_similarity, open('latent_similarity.pkl', 'wb'))

# Load movies and latent_similarity from pickle files
movies = pickle.load(open('moviess.pkl', 'rb'))
latent_similarity = pickle.load(open('latent_similarity.pkl', 'rb'))

['Star Wars: Episode III - Revenge of the Sith', 'Stargate', 'Insurgent', 'Godzilla: Final Wars', 'Superman II: The Richard Donner Cut']
